In [11]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI
# simple sequential chain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

import os

In [12]:
os.environ["AZURE_OPENAI_API_KEY"] = "14cb551b1ce5493084f689735ac91281"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://caecoai0jiaoa01.openai.azure.com/"

In [123]:
with open('DWSD0612_completo_V3.txt', 'r') as file:
    data = file.read().replace('\n', '').strip()
    data = ' '.join(data.split())

In [203]:
final = []
divisions = ["ENVIRONMENT DIVISION", "DATA DIVISION","PROCEDURE DIVISION"]
data_split = data.split('DIVISION')[2:]
for i,row in enumerate(data_split):
    final.append(f'{divisions[i]} {row}')


In [204]:
final

['ENVIRONMENT DIVISION . CONFIGURATION SECTION. SPECIAL-NAMES. DECIMAL-POINT IS COMMA. INPUT-OUTPUT SECTION. FILE-CONTROL. SELECT ARQPARM ASSIGN TO ARQPARM. SELECT ARQSORT ASSIGN TO ARQSORT. SELECT DIMESTIP ASSIGN TO DIMESTIP. SELECT ARQFTP ASSIGN TO ARQFTP. DATA ',
 "DATA DIVISION . FILE SECTION. FD ARQPARM BLOCK CONTAINS 0 RECORDS RECORDING MODE IS F LABEL RECORD IS STANDARD. 01 REG-PARAMETRO PIC X(80). SD ARQSORT. 01 REG-ARQSORT. 03 NUM-CGC-SORT PIC 9(12). 03 DATA-CANCEL-SORT PIC 9(08). 03 DATA-INIC-SORT PIC 9(08). 03 FILLER REDEFINES DATA-INIC-SORT. 05 ANO-INIC-SORT PIC 9(04). 05 MES-INIC-SORT PIC 9(02). 05 DIA-INIC-SORT PIC 9(02). 03 NUM-DV-CGC-SORT PIC 9(02). 03 COD-CIA-SORT PIC 9(03). 03 COD-APOLICE-SORT PIC 9(07). 03 NOME-ESTIP-SORT PIC X(35). 03 COD-ATIV-SORT PIC 9(04). FD DIMESTIP BLOCK CONTAINS 0 RECORDS RECORDING MODE IS F LABEL RECORD IS STANDARD. 01 REG-ESTIPULANTE PIC X(68). FD ARQFTP BLOCK CONTAINS 0 RECORDS RECORD CONTAINS 80 RECORDING MODE IS F LABEL RECORDS IS STANDA

In [49]:
model = AzureChatOpenAI(
    api_version="2023-05-15",
    azure_deployment="EY_FSO_ChatTest",
    temperature=0.1
)
model_1 = AzureChatOpenAI(
    api_version="2023-05-15",
    azure_deployment="EY_FSO_ChatTest",
    temperature=0.7
)

In [116]:
code_analysis = """As an experienced software engineer focused on the Cobol programming language, your expertise lies in decoding and elucidating every single piece of Cobol code.

Your task is to explain meticulously the provided Cobol code. Provide a thorough analytical explanation of the program's logic. The goal is to make comprehensible the code's functionality in depth.

explain the followig Cobol code:
{code}

Make sure to provide an in-depth explanation of each line of code in the broader context of the entire script. Always keep in mind, the main goal here is to reveal the intricacies of the embedded business logic within the code. 

# Pay extra attention to detailing the logic within the 'procedure division' and "data division" in the code.
# Confirm that all aspects of the code are elucidated intricately, laying bare each procedure, function, and routine.

We expect the explanation of the code logic to fall within the following steps:
   1. Identification division
   2. Environment division
   3. Data division
        1. File section
        2. Working storage section
        3. Linkage section
        4. Report section
        5. File control section
        6. I-O control section
        7. Sort-merge file section
   4. Procedure division
        1. Start of the code
        2. Reading of files and tables (Explain all the details of the files and tables used in the code.)
        3. Data filtration based on established business rules (Explain the decision parameters and conditions.)
        4. Data sorting (Explain the sorting logic considering ascending and descending order.)
        5. Extracting all queries SQL (Explain all statements like, SELECT, INSERT, UPDATE, DELETE, WHERE, ORDER BY, GROUP BY, HAVING, etc.)
        6. Writing data to output files
        7. Writing of header and trailer records        
   5. End of the code

# The explanation should be detailed and comprehensive, providing a clear understanding of the code's logic and functionality.
# The logic within a COBOL program is not inherently organized into command blocks; however, we can simulate the execution of a sequence of commands using the PERFORM command. The PERFORM command in COBOL initiates a sequence of commands that start at the beginning of a specified paragraph, which is situated directly after the PERFORM expression. If a second paragraph name follows the PERFORM command, that signifies the end of the command sequence.
# This Program, execution is diverted to the initial paragraph and concludes at the paragraph that ends the set of instructions that must be executed in structured programs. It is coupled with an EXIT command and the name of the concluding paragraph at the end of the block. This EXIT command terminates the execution of the command sequence and allows control to return to the command following the Perform command that instigated the sequence.
"""


code_analysis = """Given the following piece of code, I need you to provide a comprehensive, step-by-step explanation of its functionality. Break down each part of the code, explaining the purpose of each line, what it does, and how each piece functions within the larger program context. Please provide a detailed analysis of functions, variables, loops, and conditionals used —explain their roles, how they work, their inputs and outputs, and the implications of changing their values. This will help me enhance my understanding of the code, identify opportunities for optimization, fix potential bugs, and conduct effective testing. Your insights could also suggest how the code might behave under different scenarios or conditions.

code:
{code}

"""

# 1. Reading of files and tables (Include all the details of the files and tables used in the code.)
#         2. Data filtration based on established business rules (Include the decision parameters and business rules.)
#         3. Data sorting
#         5. Extracting all queries SQL (Include all statements like, SELECT, INSERT, UPDATE, DELETE, WHERE, ORDER BY, GROUP BY, HAVING, etc.)
#         4. Writing data to output files
#         6. Writing of header and trailer records

converter = """You're an expert software engineer in PySpark programming languages. your task is, based on a explaination step by step of logical program, you need to write PySpark code.

based on the following explanation You should write a PySpark code: 
{explanation}



"""

converter = """You're an expert software engineer. Your task is to convert the cobol program into pyspark.

Use the following explanation of the code to support and make the requirements easier to understand:
{explanation}

Use the following COBOL code and convert it to Pyspark:
{code}

Performing the following steps:
1 - In the Identification Division obtain information about the name of the program and the author as well as the comments
2 - In the environment division obtain the information of the VSAM and sequential files that will be opened and used by the program and the way to use them for input output
3 - In the data division get the structures of the fields that are defined in the files of the enviroment division
4 - In the working Storage section, get the SQL commands that define the cursors that will be used in the procedure division.
5 - In the working storage section, get the fields that will be used in the procedure division.
6 - In the procedure division, consider the entire process of reading files, writing files, sorting, sql execution deciding, moving cursors, moving fields, opening, and closing files and cursors.
"""


prompt = PromptTemplate(
    input_variables=["code"],
    template=code_analysis
)

second_prompt = PromptTemplate(
    input_variables=["explanation","code"],
    template=converter,
    )


chain = LLMChain(llm=model_1, prompt=prompt, output_key='explanation')
chain_two = LLMChain(llm=model, prompt=second_prompt, output_key='script')
overall_chain = SequentialChain(chains=[chain, chain_two],input_variables=["code"],verbose=True)


In [113]:
with get_openai_callback() as cb:
    python_script = chain.run(code=data)
print(cb.total_tokens)
print((cb.total_tokens/1000)*0.12)

16073
1.92876


In [114]:
print(python_script)

This is a COBOL (Common Business Oriented Language) program, which is a procedural language used in business and finance systems, especially in legacy systems. COBOL is known for its English-like syntax which makes the code self-documenting and highly readable.

This particular program is divided into several sections.

1. IDENTIFICATION DIVISION: This part of the program provides some basic information about the program, such as its name (DWSD0612) and author (CLAUDIA MARIA).

2. ENVIRONMENT DIVISION: This part of the program defines the computer environment in which the program will run. Here, it is specifying that the decimal point is a comma.

3. INPUT-OUTPUT SECTION: This part of the program sets up files for input and output. There are multiple file descriptions (FD) defined here, such as ARQPARM, ARQSORT, DIMESTIP, and ARQFTP.

4. DATA DIVISION: This part of the program is where data items and data structures are defined. It includes a FILE SECTION for defining the format of the

In [117]:
with get_openai_callback() as cb:
    out = chain_two.run(explanation=python_script,code=data)
print(cb.total_tokens)
print((cb.total_tokens/1000)*0.12)

16634
1.99608


In [104]:
# with get_openai_callback() as cb:
#     final_out = chain_three.run(code=data,script=out)
# print(cb.total_tokens)
# print((cb.total_tokens/1000)*0.12)

In [119]:
with open("Outputs/Cobol2spark_24.md", "w") as text_file:
    text_file.write("Explaination of Cobol code and PySpark code:\n\n")
    text_file.write(python_script)
    text_file.write("\n\n")
    text_file.write("PySpark code:\n\n")
    text_file.write(out)

In [24]:
code_analysis = """You're a software engineer specialized in Cobol programming language. You possess the necessary skills and expertise to decode and break down every single line of Cobol code.

Your task is to examine the provided Cobol code and translate to PySpark code, Ensuring that all functions is covered.

Translate the following Cobol code into PySpark code: 
{code}

"""


prompt = PromptTemplate(
    input_variables=["code"],
    template=code_analysis
)


chain = LLMChain(llm=model, prompt=prompt, output_key='explanation')



In [205]:
from tqdm import tqdm
responses = []
for row in tqdm(final):
    responses.append(chain.run(code=row))
    #python_script = chain.run(code=data)

In [32]:
with open("Outputs/Cobol2Spark_10.txt", "w") as text_file:
    text_file.write(out)

In [7]:
code_analysis = """"As an experienced software engineer focused on the Cobol programming language, your expertise lies in decoding and elucidating every single bite of Cobol code.

Your task entails meticulously dissecting the provided Cobol code. You are to provide a thorough analytical explanation of the program's logic. The goal is to make comprehensible the code's functionality, the methodology employed, and the overarching objectives it seeks to accomplish.

Cobol code: 
{code}

Make sure to provide an in-depth explanation of each line of code in the broader context of the entire script. Always keep in mind, the main goal here is to reveal the intricacies of the embedded business logic within the code. 

# Pay extra attention to detailing the logic within the 'procedure division' section of the code.
# Confirm that all aspects of the code are elucidated intricately, laying bare each procedure, function, and routine.

We expect the explanation of the code logic to fall within the following steps:
   1. Identification division
   2. Environment division
   3. Data division
   4. Procedure division
      1. Reading of files and tables
      2. Data filtration based on established business rules
      3. Data sorting
      4. Writing data to output files
      5. Writing of header and trailer records
   5. End of the code"

Remember, this process is not about explaining Cobol's inherent logic but about the specific code's functionality and purpose.
"""



converter = """You're an expert software engineer specialized in Cobol programming language. 
Your task is to convert the cobol program into pyspark.
Use the following explanation of the code to support and make the requirements easier to understand:
{explanation}
Use the following COBOL code and convert it to Pyspark by following the steps:
{code}
Performing the following steps:
1 - In the Identification Division obtain information about the name of the program and the author as well as the comments
2 - In the environment division obtain the information of the VSAM and sequential files that will be opened and used by the program and the way to use them for input output
3 - In the data division get the structures of the fields that are defined in the files of the enviroment division
4 - In the working Storage section, get the SQL commands that define the cursors that will be used in the procedure division.
5 - In the working storage section, get the fields that will be used in the procedure division.
6 - In the procedure division, consider the entire process of reading files, writing files, sorting, sql execution deciding, moving cursors, moving fields, opening, and closing files and cursors.
"""


verifier = """ Given the following code in COBOL and PySpark, Verify if the PySpark code is correct and if the explanation is correct.

COBOL code:
{script}

PySpark code:
{code}

# Assure that all the functions are covered in the PySpark code.
# Assure that all Filters, Joins, and Aggregations are accurately translated.
# assure that the PySpark code is accurate and functional.
# Assure that the PySpark code is correctly translated from the COBOL code.
"""

prompt = PromptTemplate(
    input_variables=["code"],
    template=code_analysis
)

second_prompt = PromptTemplate(
    input_variables=["explanation","code"],
    template=converter,
    )

third_prompt = PromptTemplate(
    input_variables=["script","code"],
    template=verifier,
    )



chain = LLMChain(llm=model_1, prompt=prompt, output_key='explanation')
chain_two = LLMChain(llm=model, prompt=second_prompt, output_key='script')
chain_three = LLMChain(llm=model, prompt=third_prompt, output_key='final_code')

overall_chain = SequentialChain(chains=[chain, chain_two],input_variables=["code"],verbose=True)


In [8]:
with get_openai_callback() as cb:
    python_script = overall_chain.run(code=data)


c:\Users\UN933GX\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


In [9]:
print(cb.total_tokens)
print((cb.total_tokens/1000)*0.06)

33412
2.00472


In [10]:
with open("Outputs/Cobol2Spark_13.txt", "w") as text_file:
    text_file.write(python_script)

In [79]:
print(python_script)

The COBOL program you have provided appears to be a data processing program that reads from a database, applies certain rules for filtering and sorting the data, and then writes the processed data to various output files. Here's a detailed explanation of the program, broken down into the four divisions:

1. Identification division: The program is identified by the ID DWSD0612 and is authored by Claudia Maria. This section is used to provide basic details about the program.

2. Environment division: This section is used to specify the machine and operating system the program will run on. It also defines the files that will be used in the program, which are ARQPARM, ARQSORT, DIMESTIP, and ARQFTP.

3. Data division: This is where all the data items used in the program are declared. It includes the structures of the files defined in the environment division (ARQPARM, ARQSORT, DIMESTIP, and ARQFTP), as well as other working storage variables and variables for SQL operations. The data divisi

In [62]:
len(data)/4

21070.0

In [63]:
x = data
chunks, chunk_size = len(x), 4
len([ x[i:i+chunk_size] for i in range(0, chunks, chunk_size) ])

21070

In [64]:
print(cb.total_tokens)
print((cb.total_tokens/1000)*0.06)

33013
1.9807799999999998


In [65]:
print((cb.total_tokens/1000)*0.01)

0.33013
